In [3]:
import pandas as pd
import numpy as np

import os
import pathlib
from pathlib import Path

import cv2 
from PIL import Image 
from tensorflow.keras.preprocessing import image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator #imagedatagenerater는 이미지를 학습시킬 때 학습 데이터의 양이 적을 경우 학습데이터를 조금씩 변형 시켜서 학습데이터의 양을 늘리는 방식중 하나입니다.
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')# IPython 에서 제공하는 Rich output 에 대한 표현 방식입니다.

In [7]:
data_dir = pathlib.Path('./gtsrb-german-traffic-sign/Meta')
train_path = pathlib.Path('./gtsrb-german-traffic-sign/Train')
test_path = pathlib.Path('./gtsrb-german-traffic-sign/Test')

# 전처리: 이미지 별로 사이즈가 다르기 때문에 이미지의 폭과 높이를 같은 크기로 통일합니다.
IMG_HEGHT = 30
IMG_WIDTH = 30
channels = 3

# 현재 디렉토리 위치 변경합니다.
os.chdir("/Users/jsh/.kaggle")
Path.cwd()

# 카테고리 수 확인합니다.
NUM_CATEGORIES = len(os.listdir(data_dir))
NUM_CATEGORIES

45